In [1]:
# package
'''
python-dotenv
langchain
langchain-openai

'''

'''
참고 자료 : LangChain Documents
'''

'\n참고 자료 : LangChain Documents\n'

OPENAI_API_KEY 설정

In [4]:
import os

# 현재 환경에서 실행 경로와 ipynb 파일 경로가 달라 환경 경로 일치
new_directory = './workspace/RAG'
os.chdir(new_directory)

In [5]:
# API 키를 환경변수로 관리하기 위한 설정 파일
# .env 파일에 OPENAI_API_KEY="My_OPENAI_API_KEY" 작성

from dotenv import load_dotenv

load_dotenv()

True

Langchain으로 구축한 애플리케이션은 여러 단계에 걸쳐 LLM 호출을 여러 번 사용하게 됨. <br>
이러한 어플이 점점 더 복잡해짐에 따라, 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사할 수 있는 능력이 매우 중요해짐. <br>
이를 위한 최선의 방법은 LangSmith를 사용하는 것. <br>

첫 번째 RAG 기반 QA 봇

첫 번째 튜토리얼에는 네이버 뉴스기사의 내용에 대해 질문할 수 있는 뉴스기사 QA 앱을 구축함
이 가이드에서는 OpenAI 챗 모델과 임베딩, 그리고 Chroma 벡터 스토어를 사용
하지만 여기서 보여지는 모든 것은 어떤 ChatModel이나 LLM, 임베딩, 그리고 VectorStore 또는 Retriever와도 작동함.
먼저 다음의 과정을 통해 간단한 인덱싱 파이프라인과 RAG 체인을 약 20줄의 코드로 구현할 수 있음.

라이브러리
bs4,
langchain : RecursiveCharacterTextSPlitter, 문서 로딩(WebBaseLoader), 벡터 저장(Chroma, FAISS)<br>
출력 파싱(StrOutputParser), 실행 가능한 패스스루


In [4]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./workspace/llm.md")
raw_doc = loader.load()

In [5]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

In [6]:
[e.value for e in Language]

['cpp',
 'go',
 'java',
 'kotlin',
 'js',
 'ts',
 'php',
 'proto',
 'python',
 'rst',
 'ruby',
 'rust',
 'scala',
 'swift',
 'markdown',
 'latex',
 'html',
 'sol',
 'csharp',
 'cobol',
 'c',
 'lua',
 'perl']

In [7]:
RecursiveCharacterTextSplitter.get_separators_for_language(Language.PYTHON)

['\nclass ', '\ndef ', '\n\tdef ', '\n\n', '\n', ' ', '']

In [8]:
PYTHON_CODE = """
def hello_world():
    print("Hello, World!")

# Call the function
hello_world()
"""
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs

[Document(page_content='def hello_world():\n    print("Hello, World!")'),
 Document(page_content='# Call the function\nhello_world()')]

In [9]:
markdown_text = """
# 🦜️🔗 LangChain

⚡ Building applications with LLMs through composability ⚡

## Quick Install

```bash
# Hopefully this code block isn't split
pip install langchain
```

As an open-source project in a rapidly developing field, we are extremely open to contributions.
"""

In [10]:
md_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN, chunk_size=60, chunk_overlap=0
)
md_docs = md_splitter.create_documents([markdown_text])
md_docs

[Document(page_content='# 🦜️🔗 LangChain'),
 Document(page_content='⚡ Building applications with LLMs through composability ⚡'),
 Document(page_content='## Quick Install\n\n```bash'),
 Document(page_content="# Hopefully this code block isn't split"),
 Document(page_content='pip install langchain'),
 Document(page_content='```'),
 Document(page_content='As an open-source project in a rapidly developing field, we'),
 Document(page_content='are extremely open to contributions.')]

In [44]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('your openai api key')

In [46]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import FAISS

# Load the document, split it into chunks, embed each chunk and load it into the vector store.
import os
openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ['OPENAI_API_KEY'] = "your openai api key"
raw_documents = TextLoader("llm.md").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings(openai_api_key=openai_api_key))

In [49]:
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(api_key=openai_api_key)

embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

In [50]:
embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")
embedded_query[:5]

[0.005384807424727807,
 -0.0005522561790177147,
 0.03896066510130955,
 -0.002939867294003909,
 -0.008987877434176603]